In [1]:
import pandas as pd
import numpy as np


In [2]:
data = pd.read_csv('novo_arquivo.csv')

In [4]:
unique_values = data['situacao'].unique()
print(unique_values)


['Ativo' 'Inativo' nan]


In [5]:
data

,cdn_cliente,num_aval1,num_aval2,dt_inscricao,l_cpf,estrutura,Bloqueado,log_ativo,mot_ativo,cdn_repres,...,data-ativacao,data-inativacao,log_ativo_pa,log_cliente_pet,cod_rep_secundario,apl_classif_produtor,apl_dt_valid_classif,cod_transp,log_participante_bayer,log_cooperativa
0,1,0,0,1999-11-01,False,Médio,False,True,COOP Nº 6502 - 23/07/2009,35006,...,2015-12-10,(null),True,True,19069,1.0,9999-12-31,71214,False,False
1,2,0,0,1996-03-01,False,Mini,False,True,NaN,28000,...,(null),(null),False,False,0,1.0,9999-12-31,71215,False,False
2,3,0,0,1996-01-24,False,Grande1,False,True,NaN,20000,...,(null),(null),False,False,0,1.0,9999-12-31,48549,False,False
3,4,0,0,1996-03-04,False,Grande1,False,True,NaN,49010,...,2015-09-29,(null),True,False,0,50.0,9999-12-31,1241,False,False
4,5,0,0,1996-01-24,False,Grande1,False,True,NaN,23100,...,(null),(null),False,False,0,1.0,9999-12-31,71216,False,False
